# Neural Nets 2
Ben Jafek  
MATH 404  
2/26/18

In [10]:
import time
import numpy as np
from matplotlib import pyplot as plt 

from keras.datasets import fashion_mnist as fmn
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import to_categorical

# Problem 1
Experiment with fully connected neural nets for classification of the Fashion-MNIST data: add at least two more layers, make all hidden layers at least 20 neurons wide, and try it with both ReLU and sigmoid activations.  Train for as many epochs as you need until the loss function (categorical cross entropy) stops improving--Keras's `callbacks.EarlyStopping` may be useful.

In [2]:
(x_train, y_train), (x_test, y_test) = fmn.load_data()
#Flatten the inputs to L*784 instead of L*28*28
x_train = x_train.reshape(len(x_train), 28*28)
x_test = x_test.reshape(len(x_test), 28*28)

# Convert labels to categorical one-hot encoding
one_hot_train_labels = to_categorical(y_train, num_classes=10)
one_hot_test_labels = to_categorical(y_test, num_classes=10)

In [9]:
ACCS = []
TIMES = []
num_epochs = 30
ovr_start = time.time()

for act in ['relu', 'sigmoid']:
    print (act)
    
    model = Sequential()
    model.add(Dense(64, input_dim=784))
    model.add(Activation(act))

    model.add(Dense(32, input_dim=64))
    model.add(Activation(act))

    model.add(Dense(32, input_dim=32))
    model.add(Activation(act))

    model.add(Dense(16, input_dim=32))
    model.add(Activation(act))

    model.add(Dense(10, input_dim=16))
    model.add(Activation('softmax'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    # Train the model, iterating on the data in batches of 32 samples
    start = time.time()
    hist = model.fit(x_train, one_hot_train_labels, epochs=3, batch_size=32, verbose=0)
    elapsed = time.time() - start
    
    #Now save it for future indulgence.
    acc_arr = np.array(hist.history['acc']) 
    loss_arr = np.array(hist.history['loss'])
    time_arr = np.array(elapsed)
    np.save('NN2/{}_accs.npy'.format(act), acc_arr)
    np.save('NN2/{}_time.npy'.format(act), time_arr)
    np.save('NN2/{}_loss.npy'.format(act), loss_arr)
    
print ('TOTAL TIME ELAPSED: {:.3f}'.format(time.time() - ovr_start))

relu
sigmoid


# Problem 2
Using the notation from class today (and from the video) calculate one iteration of backpropagation by hand (or code something to do it for you).  That is, calculate the forward pass and then the backward pass to compute both the output of the network for the current weights, and the gradient (with respect to the Ws and the bs) on a (fully connected) neural network with two hidden layers of 2 neurons each (ReLu activation), two inputs, and a single output layer having a sigmoid activation function. Use the input data $x=(1,-1)$, $y=1$, and assume the current weights are

W^1 = 0.25  0.1
            -0.2   0.9

b^(1) =   0.1
             -0.2

W^(2) =  0.5   0.8
               0.3   0.7

b^(2) =  -0.3
               0.1

W^(3) =  0.1   -0.2

b^(3) = 0.3$

So the structure of the network looks something like this:

            L_1      L_2      L_3
            
    x_0 ----> O ----> O ----> O ----> 

        \  /        \  /           /
       
         / \         / \         /
        
          x_1 ----> O ----> O

where the Os here represent neurons